In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

In [2]:
# Fetch the dataset
rt_iot2022 = fetch_ucirepo(id=942)
X = rt_iot2022.data.features
y = rt_iot2022.data.targets

In [3]:
# Combine X and y into a single DataFrame
df = pd.concat([X, y], axis=1)

# Encode the 'Attack_type' column using LabelEncoder
encoder = LabelEncoder()
df['Attack_type'] = encoder.fit_transform(df.iloc[:, -1])

# Identify categorical columns and one-hot encode them
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
df = pd.get_dummies(df, columns=categorical_cols)

# Split the data into features and labels
X = df.drop('Attack_type', axis=1).values
y = df['Attack_type'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# Split data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Convert datasets to TensorFlow datasets
def create_dataset(X, y, batch_size=64, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1024)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = create_dataset(X_train, y_train, shuffle=True)
val_dataset = create_dataset(X_val, y_val)
test_dataset = create_dataset(X_test, y_test)

In [8]:
def build_model(num_features, num_classes):
    model = Sequential([
        Input(shape=(num_features,)),  # Define the input shape explicitly
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [9]:
# Build and compile the model
num_features = X_train.shape[1]
num_classes = len(np.unique(y))
model = build_model(num_features, num_classes)
model.compile(optimizer=Adam(0.001), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Training the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=20)

Epoch 1/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9207 - loss: 0.3132 - val_accuracy: 0.9889 - val_loss: 0.0430
Epoch 2/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9855 - loss: 0.0451 - val_accuracy: 0.9910 - val_loss: 0.0343
Epoch 3/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9883 - loss: 0.0368 - val_accuracy: 0.9923 - val_loss: 0.0267
Epoch 4/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9890 - loss: 0.0369 - val_accuracy: 0.9937 - val_loss: 0.0240
Epoch 5/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9903 - loss: 0.0308 - val_accuracy: 0.9938 - val_loss: 0.0236
Epoch 6/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9918 - loss: 0.0263 - val_accuracy: 0.9933 - val_loss: 0.0234
Epoch 7/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9927 - loss: 0.0226 - val_accuracy: 0.9946 - val_loss: 0.0195
Epoch 8/20
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9926 - loss: 0.0228 - 

In [10]:
model.save('Tensorflow_IOT_model.keras')

In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

385/385 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.9939 - loss: 0.0251
Test Loss: 0.0313, Test Accuracy: 0.9944


In [14]:
def evaluate_class_accuracy(model, test_dataset):
    y_true = []
    y_pred = []

    # Collect all labels and predictions for the test set
    for batch in test_dataset:
        x, labels = batch
        logits = model(x)
        predictions = np.argmax(logits, axis=1)
        y_true.extend(labels.numpy())
        y_pred.extend(predictions)

    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    class_accuracy = cm.diagonal() / cm.sum(axis=1)  # Accuracy for each class
    return class_accuracy

class_accuracies = evaluate_class_accuracy(model, test_dataset)
for idx, acc in enumerate(class_accuracies):
    print(f"Accuracy of Class {idx} ({labels[idx]}): {acc:.4f}")

2024-04-15 17:26:04.582102: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


NameError: name 'labels' is not defined

In [ ]:
# Extract loss and validation loss from history
train_losses = history.history['loss']
val_losses = history.history['val_loss']

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('train_val_loss.svg', format='svg')
plt.show()

In [ ]:
# Assuming class_accuracies are calculated and available
classes = list(range(len(class_accuracies)))  # Update this if you have specific class names

plt.figure(figsize=(12, 6))
plt.bar(classes, class_accuracies, color='skyblue')
plt.xlabel('Class Labels')
plt.ylabel('Accuracy')
plt.title('Accuracy for Each Class')
plt.xticks(ticks=classes)  # Ensure proper class label names if available
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.savefig('class_accuracies.svg', format='svg')
plt.show()